In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
%pwd

'/Users/lakshayarora/Desktop/Projects/Text-Summary-Generator'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [5]:
from textSummaryGenerator.constants import *
from textSummaryGenerator.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            # keep tokenizer aligned with stage-04 trainer model
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
import os
from textSummaryGenerator.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

/Users/lakshayarora/Desktop/Projects/Text-Summary-Generator/proj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self, example_batch):

        model_inputs = self.tokenizer(
            ["summarize: " + dialogue for dialogue in example_batch["dialogue"]],
            max_length=1024,
            truncation=True,
            padding="max_length"   # important for batching
        )

        labels = self.tokenizer(
            text_target=example_batch["summary"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        model_inputs["labels"] = labels["input_ids"]

        return model_inputs

    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [12]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2026-02-15 20:07:13,197: INFO: common: yaml file: config/config.yaml loaded successfully]
[2026-02-15 20:07:13,198: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-15 20:07:13,199: INFO: common: created directory at: artifacts]
[2026-02-15 20:07:13,200: INFO: common: created directory at: artifacts/data_transformation]
[2026-02-15 20:07:13,353: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-15 20:07:13,382: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-15 20:07:13,508: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-15 20:07:13,522: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-c

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 222889.67 examples/s]
